In [1]:
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder
from operator import attrgetter
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from tqdm import tqdm
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
import pickle

def runModel(model,bill):
    test_data = bill.drop(['bill_number','bill_type','index','result','status'],axis=1)
    X = (test_data.as_matrix().flatten()).reshape(1,-1)
    result = model.predict(X)
    return result

def modelConf(model,bill):
    test_data = bill.drop(['bill_number','bill_type','index','result','status'],axis=1)
    X = (test_data.as_matrix().flatten()).reshape(1,-1)
    P = model.predict_proba(X)
    Pr = max(P[0])*100. #convert to percent
    return Pr

def getResult(model,bill_type,bill_number,engine):
    bill = retrieveFeatures(bill_type,bill_number,engine)
    output = np.zeros((2))
    output[0] = runModel(model,bill)
    output[1] = modelConf(model,bill)
    return output

def retrieveFeatures(bill_type,bill_number,engine):
    #pull a bill's features from postgresql server
    #query = "SELECT * FROM features WHERE bill_type LIKE '"+bill_type+"' AND bill_number LIKE '"+bill_number+"';"
    query = "SELECT * FROM features WHERE bill_type LIKE '"+bill_type+"' AND bill_number LIKE '"+bill_number+"' LIMIT 1;"
    bill = pd.read_sql_query(query,engine) #note to self, bills repeat, I only want the most recent one.
    return bill

Populating the interactive namespace from numpy and matplotlib


In [2]:
#set up postgresql engine
dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)

#load model
#model_file = 'logreg_model_subjects.pkl' #logistic regression
model_file = 'random_forest_model_subjects.pkl' #random forest
mfile = open(model_file,'rb')
model = pickle.load(mfile)
mfile.close()

In [3]:
bill_type = str(input("Please enter originating chamber: "))
bill_number = str(input("Please enter bill number: "))
result = getResult(model,bill_type,bill_number,engine)
if result[0] == 1:
    print("Predicted Result: PASS")
if result[0] == 0:
    print("Predicted Result: FAIL")
print("Confidence of Prediction: ",result[1])

Please enter originating chamber: hr
Please enter bill number: 4387
Predicted Result: PASS
Confidence of Prediction:  82.0
